In [ ]:
from pprint import pprint
import json
import ast
from collections import defaultdict

import numpy as np

with open('heur_results/evaluate_heur_mnist.txt') as f:
    lines = f.readlines()

In [ ]:
heur_results = defaultdict(list)
accs = defaultdict(list)

for line in lines:
    if line[0].isdigit and line[1] == " ":
        v = int(line[0])
        heur_params = ast.literal_eval(line[2:])

        heur_results[v].append(heur_params)

        accs[v].append([heur_params["fedavg_acc"], heur_params["heur_acc"]])

In [ ]:
for v, vaccs in accs.items():
    print(f"Heuristic v{v}")
    vaccs_arr = np.array(vaccs)
    diff = vaccs_arr[:, 1] - vaccs_arr[:, 0]
    mean_acc = np.mean(vaccs_arr, axis=0)
    print(f"\tMean FedAvg: {mean_acc[0]}")
    print(f"\tMean Heuristic: {mean_acc[1]}")
    print(f"\tMean difference: {np.mean(diff)} ± {np.std(diff)}")

In [ ]:
with open("heur_results/heur_svhn.json", 'w') as outfile:
    json.dump(heur_results, outfile)

for v, vaccs in accs.items():
    print(f"Heuristic v{v}")
    vaccs_arr = np.array(vaccs)
    diff = vaccs_arr[:, 1] - vaccs_arr[:, 0]
    mean_acc = np.mean(vaccs_arr, axis=0)
    print(f"\tMean FedAvg: {mean_acc[0]}")
    print(f"\tMean Heuristic: {mean_acc[1]}")
    print(f"\tMean difference: {np.mean(diff)} ± {np.std(diff)}")

In [ ]:
with open("heur_results/heur_cifar10.json", 'w') as outfile:
    json.dump(heur_results, outfile)

for v, vaccs in accs.items():
    print(f"Heuristic v{v}")
    vaccs_arr = np.array(vaccs)
    diff = vaccs_arr[:, 1] - vaccs_arr[:, 0]
    mean_acc = np.mean(vaccs_arr, axis=0)
    print(f"\tMean FedAvg: {mean_acc[0]}")
    print(f"\tMean Heuristic: {mean_acc[1]}")
    print(f"\tMean difference: {np.mean(diff)} ± {np.std(diff)}")

In [ ]:
import json

with open("heur_results/heur_results_emnist_2022-01-22 11:15:54.170890.json", 'r') as f:
  emnist01 = json.load(f)

with open("heur_results/heur_results_emnist_2022-01-26 07:55:26.604491.json", 'r') as f:
  emnist2 = json.load(f)

with open("heur_results/heur_results_emnist_2022-01-26 08:09:56.874550.json", 'r') as f:
  emnist3 = json.load(f)

In [ ]:
with open("heur_results/heur_emnist.json", "w") as fout:
    json.dump(emnist01 | emnist2 | emnist3, fout)